In [23]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2DTranspose

In [24]:
class CustomDataset(tf.keras.utils.Sequence):
    def __init__(self, images_dir, annotations_dir, batch_size=32, image_size=(128, 128), shuffle=True):
        self.images_dir = images_dir
        self.annotations_dir = annotations_dir
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle

        self.image_paths = sorted(os.listdir(self.images_dir))
        self.annotation_paths = sorted(os.listdir(self.annotations_dir))

        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]

        batch_images = []
        batch_annotations = []

        for i in batch_indexes:
            image_path = os.path.join(self.images_dir, self.image_paths[i])
            annotation_path = os.path.join(self.annotations_dir, self.annotation_paths[i])

            image = cv2.imread(image_path)
            annotation = cv2.imread(annotation_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, self.image_size)
            annotation = cv2.resize(annotation, self.image_size)
            image = image.astype(np.float32) / 255.0

            batch_images.append(image)
            batch_annotations.append(annotation)

        return np.array(batch_images), np.array(batch_annotations)

In [25]:
images_dir = r"C:\Users\dharu\OneDrive\Desktop\6th Semester\Machine Learning\test\images\test\t5"
annotations_dir = r"C:\Users\dharu\OneDrive\Desktop\6th Semester\Machine Learning\test\annotations\test\t5"

In [30]:
train_dataset = CustomDataset(images_dir, annotations_dir, batch_size=32, image_size=(128, 128), shuffle=True)

In [31]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2DTranspose

In [32]:
def create_segmentation_model(input_shape, num_classes):
    base_model = ResNet50(input_shape=input_shape, include_top=False)
    x = base_model.output
    x = Conv2DTranspose(256, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = Conv2DTranspose(128, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = Conv2DTranspose(num_classes, kernel_size=(1, 1), padding='same', activation='softmax')(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    return model

In [33]:
batch_size = 32
image_size = (128, 128)  
input_shape = (image_size[0], image_size[1], 3)  
num_classes = 3
segmentation_model = create_segmentation_model(input_shape, num_classes)

In [34]:
segmentation_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [35]:
segmentation_model.fit(train_dataset, epochs=10)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert defined at (most recent call last):
  File "C:\Users\dharu\anaconda3\lib\runpy.py", line 197, in _run_module_as_main

  File "C:\Users\dharu\anaconda3\lib\runpy.py", line 87, in _run_code

  File "C:\Users\dharu\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\dharu\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance

  File "C:\Users\dharu\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start

  File "C:\Users\dharu\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start

  File "C:\Users\dharu\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever

  File "C:\Users\dharu\anaconda3\lib\asyncio\base_events.py", line 1905, in _run_once

  File "C:\Users\dharu\anaconda3\lib\asyncio\events.py", line 80, i